# Lump Experiements
Andrew E. Davidson  
aedavids@ucsc.edu  
12/28/23  


1/10/24  
starting with best10CuratedDegree1_ce467ff stage results, fixing mis classification errors between LUAD, LUSC, and Lung is not going to be good enough for deconvolution. How ever given we use only 20 genes will probably be good enough for elife binary classifier

```
root="/private/groups/kimlab/aedavids/deconvolution/1vsAll~gender_category/"
out="best10CuratedDegree1_ce467ff/training/best10CuratedDegree1.sh.out"

fPath="${root}/${out}/metrics/metricsRounded.csv"

sensitivity tp: 268
sensitivity tp : 268 fn : 41 ret: 0.8673139158576052
 LUAD adj sensitivity = 0.8673139158576052

sensitivity tp: 15478
sensitivity tp : 15478 fn : 14 ret: 0.9990963077717532
 LUAD adj specificity = 0.9990963077717532
```

```
sensitivity tp: 219
sensitivity tp : 219 fn : 82 ret: 0.7275747508305648
LUSC adj sensitivity = 0.7275747508305648

sensitivity tp: 15429
sensitivity tp : 15429 fn : 71 ret: 0.9954193548387097
 LUSC adj specificity = 0.9954193548387097
```

## Best Bang for the buck?

```
LUAD Fn PAAD 14

LUAD FP BRCA 11
        LUSC 11

LUSC FN HNSC 21
        LUAD 11
        PAAD 19

LUSC FP BRAC 14
        HNSC 44
```

<hr />
12/28/23 results deprecated

rank15 and rank20 do not provide evidence that lumping LUAD and LUSC into a single 'lung cancer type' would improve sensitivity

## TODO AEDWIP <span style="color:red">Look a rank Lung</span>  

Looks like there is some evidenc in best100Enriched_6_Degree1GTEx_TCGA.sh.out/lungExploreClassificationErrors.sh.out that lumping might work

```
aedavids@mustard $ cat LUAD.falseNegativeGroupByCounts.csv | grep LUSC
LUSC,20
aedavids@mustard $ cat LUAD.falsePositiveGroupByCounts.csv | grep LUSC
LUSC,6

aedavids@mustard $ cat LUSC.falseNegativeGroupByCounts.csv | grep LUAD
LUAD,6
aedavids@mustard $ cat LUSC.falsePositiveGroupByCounts.csv | grep LUAD
LUAD,20
```

```
aedavids@mustard $ 
aedavids@mustard $ cat LUAD.falseNegativeGroupByCounts.csv | grep Lung
Lung,23
aedavids@mustard $ cat LUAD.falsePositiveGroupByCounts.csv | grep Lung
```

```
aedavids@mustard $ cat LUSC.falseNegativeGroupByCounts.csv | grep Lung
Lung,10
aedavids@mustard $ cat LUSC.falsePositiveGroupByCounts.csv | grep Lung
```

In [1]:
import ipynbname

# use display() to print an html version of a data frame
# useful if dataFrame output is not generated by last like of cell
from IPython.display import display

import numpy as np
import pandas as pd
# display all columns
pd.set_option('display.max_columns', None)

# import pathlib as pl
import os
import sys

In [2]:
# setting the python path allows us to run python scripts from using
# the CLI. 
ORIG_PYTHONPATH = os.environ['PYTHONPATH']

pp = ipynbname.path()
deconvolutionModules = pp.parent.joinpath("../../python")
print("deconvolutionModules: {}\n".format(deconvolutionModules))

PYTHONPATH = ORIG_PYTHONPATH + f':{deconvolutionModules}'
print("PYTHONPATH: {}\n".format(PYTHONPATH))

os.environ["PYTHONPATH"] = PYTHONPATH
PYTHONPATH = os.environ["PYTHONPATH"]
print("PYTHONPATH: {}\n".format(PYTHONPATH))

deconvolutionModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python



In [3]:
# to be able to import our local python files we need to set the sys.path
# https://stackoverflow.com/a/50155834
sys.path.append( str(deconvolutionModules) )
print("\nsys.path:\n{}\n".format(sys.path))


sys.path:
['/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning', '/private/home/aedavids/extraCellularRNA/src', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python311.zip', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/lib-dynload', '', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages/setuptools-57.4.0-py3.9.egg', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages/pip-21.1.3-py3.9.egg', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python']



In [4]:
from analysis.hyperParameterTunningMetrics import elifeCols, lungCols, findFile, findSummaryMetricsCols
from analysis.hyperParameterTunningMetrics import metricsRunner

root = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category"
notebookName = ipynbname.name()
outDir = f'{root}/hyperParameter/{notebookName}.out'
print( f'output dir: \n{outDir}' )
os.makedirs(outDir, exist_ok=True)

output dir: 
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/lump.out


In [5]:
pipelineRun = "rank20GTEx_TCGA"
parentDir = f'{root}/{pipelineRun}'
print(f' parentDir : {parentDir}' )
metricsFile = findFile(parentDir, "metricsRounded.csv")[0]
print(f' metriscFile : {metricsFile}' )

r15DF = pd.read_csv(metricsFile)
r15DF

 parentDir : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/rank20GTEx_TCGA
 metriscFile : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/rank20GTEx_TCGA/training/rank20GTEx_TCGA.sh.out/metrics/metricsRounded.csv


,id,precision,recall,f1-score,support,specificity,sensitivity,tp,fn,fp,tn
0,ACC,1.000,0.542,0.703,48.000,1.000,0.542,26.000,22.000,0.000,15753.000
1,Adipose_Subcutaneous,0.733,0.977,0.837,398.000,0.991,0.977,389.000,9.000,142.000,15261.000
2,Adipose_Visceral_Omentum,0.890,0.822,0.854,325.000,0.998,0.822,267.000,58.000,33.000,15443.000
3,Adrenal_Gland,0.966,0.903,0.933,155.000,1.000,0.903,140.000,15.000,5.000,15641.000
4,Artery_Aorta,0.948,0.977,0.962,259.000,0.999,0.977,253.000,6.000,14.000,15528.000
...,...,...,...,...,...,...,...,...,...,...,...
81,Vagina,0.753,0.649,0.697,94.000,0.999,0.649,61.000,33.000,20.000,15687.000
82,Whole_Blood,1.000,0.982,0.991,453.000,1.000,0.982,445.000,8.000,0.000,15348.000
83,accuracy,0.763,0.763,0.763,0.763,0.763,0.763,0.763,0.763,0.763,0.763
84,macro avg,0.777,0.744,0.719,15801.000,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
selectRows = r15DF.loc[ :, "id"].isin(lungCols)
r15DF.loc[selectRows, :]

,id,precision,recall,f1-score,support,specificity,sensitivity,tp,fn,fp,tn
47,LUAD,0.984,0.592,0.739,309.0,1.000,0.592,183.0,126.0,3.0,15489.0
48,LUSC,0.796,0.688,0.738,301.0,0.997,0.688,207.0,94.0,53.0,15447.0
50,Lung,0.917,0.986,0.950,347.0,0.998,0.986,342.0,5.0,31.0,15423.0


In [7]:
luadFalseNegDF = pd.read_csv(findFile(parentDir, "LUAD.falseNegativeGroupByCounts.csv")[0])
luadFalsePosDF = pd.read_csv(findFile(parentDir, "LUAD.falsePositiveGroupByCounts.csv")[0])
luscFalseNegDF = pd.read_csv(findFile(parentDir, "LUSC.falseNegativeGroupByCounts.csv")[0])
luscFalsePosDF = pd.read_csv(findFile(parentDir, "LUSC.falsePositiveGroupByCounts.csv")[0])

In [8]:
print(f'\n LUAD False Negative')
display( luadFalseNegDF[ luscFalseNegDF.predictedCat == 'LUSC' ])

print(f'\n LUAD False Positive')
display( luadFalsePosDF[ luadFalsePosDF.category == 'LUSC' ])

print(f'\n LUSC False Negative')
display( luscFalseNegDF[ luscFalseNegDF.predictedCat == 'LUAD' ])

print(f'\n LUSC False Positive')
display( luscFalsePosDF[ luscFalsePosDF.category == 'LUAD' ] )


 LUAD False Negative


/data/tmp/ipykernel_2241476/2701496115.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display( luadFalseNegDF[ luscFalseNegDF.predictedCat == 'LUSC' ])


,predictedCat,predictedCat.1



 LUAD False Positive


,category,category.1
0,LUSC,3



 LUSC False Negative


,predictedCat,predictedCat.1
17,LUAD,3



 LUSC False Positive


,category,category.1
4,LUAD,9


## calculate sensitivity and specificty for lumping LUAD and LUSC into 'lung cancer'

In [9]:
def sensitivity(
        tp : float,
        fn : float,
    ) -> float :
    print(f'sensitivity tp: {tp}')
    ret =  tp / (tp + fn)
    print(f'sensitivity tp : {tp} fn : {fn} ret: {ret}')
    return ret

def specificity(
        tn : float,
        fp : float,
    ) -> float :
    print(f'specificity tn: {tn}')    
    ret = tn / (tn + fnp)
    print(f'specificity tn : {tn} fp : {fp} ret: {ret}')
    return ret
    

def cancerLump(
    rootDir : str,
    stageName : str,
    categories : list[str],
    verbose : bool = False,
    ) :
    '''
    TODO
    '''

    path = f'{rootDir}/{stageName}'
    if verbose :
        print(f'path : {path}')
    
    metricsPaths = findFile(path, "metricsRounded.csv")
    print(f'metricsPaths : {metricsPaths}')
    metricsPath = metricsPaths[0]
    if verbose:
        print( f'\nload {stageName} :\n{metricsPath}' )
        
    metricsRawDF = pd.read_csv(metricsPath)

    selectRows = metricsRawDF.loc[:, 'id'].isin(categories)
    metricsDF = metricsRawDF.loc[selectRows, ['id', 'sensitivity', 'specificity', 'tp', 'fn', 'fp', 'tn']]
    display(metricsDF)

    # print(f' test $$$$$$$$')
    # # https://stackoverflow.com/a/56300992/4586180
    # #display( metricsDF.loc[:, ['tp', 'fn']].apply(lambda x : print(x) ) )
    # display( metricsDF.loc[:, ['tp', 'fn']].apply(lambda x : sensitivity(x.loc[:,'tp'], x.loc[:,'fn']) ) )
    # # display( metricsDF.loc[:, ['tn', 'fp']].apply(lambda x : specificity(x['tn'],x['fp']) ) )

    for c in categories:
        print(f'\n ######### category: {c}')
        
        pattern = f'{c}.falseNegativeGroupByCounts.csv'
        negativePath = findFile(path, pattern)
        negativePath = negativePath[0]
        falseNegativeDF = pd.read_csv(negativePath)
        print('category : {c} false negatives')
        selectFalseNegativeRows = falseNegativeDF.loc[:, 'predictedCat'].isin(categories)
        display(falseNegativeDF.loc[selectFalseNegativeRows, :])
                                
        pattern = f'{c}.falsePositiveGroupByCounts.csv'
        posPathPath = findFile(path, pattern)  
        posPathPath = posPathPath[0]
        falsePositiveDF = pd.read_csv(posPathPath)
        print(f'category : {c} false positives')
        selectFalsePositiveRows = falsePositiveDF.loc[:, 'category'].isin(categories)
        display(falsePositiveDF.loc[selectFalsePositiveRows, :])

        #orginal metric
        selectRows = metricsDF.loc[:, "id"] == c
        tp, fn, fp, tn = metricsDF.loc[ selectRows , ['tp', 'fn', 'fp', 'tn'] ].values[0]
        print(f'original metics category : {c} tp ; {tp} fn : {fn} fp : {fp}, tn : {tn}')

        return metricsDF

metricsDF = cancerLump(root, 
           stageName = "best10CuratedDegree1_ce467ff",
           categories=['LUAD', 'LUSC'], 
           verbose=False)

metricsPaths : ['/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1_ce467ff/training/best10CuratedDegree1.sh.out/metrics/metricsRounded.csv']


,id,sensitivity,specificity,tp,fn,fp,tn
47,LUAD,0.841,0.998,260.0,49.0,25.0,15467.0
48,LUSC,0.691,0.995,208.0,93.0,79.0,15421.0



 ######### category: LUAD
category : {c} false negatives


,predictedCat,predictedCat.1
7,LUSC,8


category : LUAD false positives


,category,category.1
2,LUSC,11


original metics category : LUAD tp ; 260.0 fn : 49.0 fp : 25.0, tn : 15467.0


In [10]:
def adjLUAD():
    # wasting to much time
    # adjLUADTP = LUADTP + LUADFN = LUSC
    adjLUADTP = 260 + 8 
    
    # adjLUADFN = LUADFN - LUADFN = LUSC
    adjLUADFN = 49 - 8
    
    # adj LUADTN = LUADTN + LUADFP = LUSC
    adjLUADTN = 15467 + 11
    
    #adj LUDFP = LUADFP - LUADFP= LUSC
    adjLUADFP = 25 - 11
    
    print(f' LUAD adj sensitivity = {sensitivity(adjLUADTP, adjLUADFN)}\n')
    print(f' LUAD adj specificity = {sensitivity(adjLUADTN, adjLUADFP)}\n')

display(metricsDF)
print()
adjLUAD()

,id,sensitivity,specificity,tp,fn,fp,tn
47,LUAD,0.841,0.998,260.0,49.0,25.0,15467.0
48,LUSC,0.691,0.995,208.0,93.0,79.0,15421.0



sensitivity tp: 268
sensitivity tp : 268 fn : 41 ret: 0.8673139158576052
 LUAD adj sensitivity = 0.8673139158576052

sensitivity tp: 15478
sensitivity tp : 15478 fn : 14 ret: 0.9990963077717532
 LUAD adj specificity = 0.9990963077717532



In [11]:
def adjLUSC():
    # wasting to much time
    # adjLUSCTP = LUSCTP + LUSCFN = LUAD
    adjLUSCTP = 208 +  11
    
    # adjLUSCFN = LUSCFN - LUSCFN = LUAD
    adjLUSCFN = 93 - 11
    
    # adj LUSCTN = LUSCTN + LUSCFP = LUAD
    adjLUSCTN = 15421 + 8
    
    #adj LUDFP = LUSCFP - LUSCFP= LUAD
    adjLUSCFP = 79 - 8
    
    print(f'LUSC adj sensitivity = {sensitivity(adjLUSCTP, adjLUSCFN)}\n')
    print(f' LUSC adj specificity = {sensitivity(adjLUSCTN, adjLUSCFP)}\n')

display(metricsDF)
print()
adjLUSC()

,id,sensitivity,specificity,tp,fn,fp,tn
47,LUAD,0.841,0.998,260.0,49.0,25.0,15467.0
48,LUSC,0.691,0.995,208.0,93.0,79.0,15421.0



sensitivity tp: 219
sensitivity tp : 219 fn : 82 ret: 0.7275747508305648
LUSC adj sensitivity = 0.7275747508305648

sensitivity tp: 15429
sensitivity tp : 15429 fn : 71 ret: 0.9954193548387097
 LUSC adj specificity = 0.9954193548387097



In [12]:
hackDir="/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1_ce467ff/training/best10CuratedDegree1.sh.out/lungExploreClassificationErrors.sh.out"
fPath = f'{hackDir}/LUAD.falsePositiveGroupByCounts.csv'
! grep 'LUAD\|LUSC\|Lung' $fPath

LUSC,11


In [13]:
fPath = f'{hackDir}/LUAD.falseNegativeGroupByCounts.csv'
! grep 'LUAD\|LUSC\|Lung' $fPath

LUSC,8
Lung,4


In [14]:
fPath = f'{hackDir}/LUSC.falsePositiveGroupByCounts.csv'
! grep 'LUAD\|LUSC\|Lung' $fPath

LUAD,8


In [15]:
fPath = f'{hackDir}/LUSC.falseNegativeGroupByCounts.csv'
! grep 'LUAD\|LUSC\|Lung' $fPath

LUAD,11
Lung,3


In [16]:
fPath = f'{hackDir}/Lung.falsePositiveGroupByCounts.csv'
! grep 'LUAD\|LUSC\|Lung' $fPath

LUAD,4
LUSC,3


In [17]:
fPath = f'{hackDir}/Lung.falseNegativeGroupByCounts.csv'
! grep 'LUAD\|LUSC\|Lung' $fPath

# hacks where can we get the biggest improvements?

```
aedavids@mustard $ for i in  `ls *GroupByCounts.csv`; do echo ""; echo $i;  cat $i; done

LUAD.falseNegativeGroupByCounts.csv
predictedCat,predictedCat
Artery_Aorta,1
Artery_Coronary,2
BLCA,2
CESC,2
CHOL,1
Colon_Sigmoid,1
HNSC,3
LUSC,8
Lung,4
MESO,1
PAAD,14
Prostate,1
READ,1
UCEC,1
UCS,6
Whole_Blood,1

LUAD.falsePositiveGroupByCounts.csv
category,category
BRCA,11
CESC,2
LUSC,11
STAD,1

Lung.falseNegativeGroupByCounts.csv
predictedCat,predictedCat
Spleen,1
Stomach,1

Lung.falsePositiveGroupByCounts.csv
category,category
Adipose_Subcutaneous,1
Adipose_Visceral_Omentum,1
Artery_Coronary,3
Brain_Amygdala,1
Brain_Anterior_cingulate_cortex_BA24,1
Brain_Hippocampus,1
Breast_Mammary_Tissue,1
Colon_Transverse,2
Esophagus_Muscularis,1
Heart_Left_Ventricle,1
LUAD,4
LUSC,3
MESO,1
Prostate,3
SARC,1
STAD,2
Small_Intestine_Terminal_Ileum,2
Uterus,1
Vagina,1

LUSC.falseNegativeGroupByCounts.csv
predictedCat,predictedCat
Artery_Coronary,2
BLCA,4
Breast_Mammary_Tissue,1
CESC,6
CHOL,2
COAD,6
Cells_Cultured_fibroblasts,1
Colon_Sigmoid,2
Esophagus_Mucosa,1
Esophagus_Muscularis,2
HNSC,21
LUAD,11
Lung,3
PAAD,19
PCPG,1
Prostate,1
Skin_Not_Sun_Exposed_Suprapubic,1
Spleen,1
UCEC,1
UCS,6
Uterus,1

LUSC.falsePositiveGroupByCounts.csv
category,category
BLCA,4
BRCA,14
CESC,2
ESCA,6
HNSC,44
LUAD,8
UCEC,1

Whole_Blood.falseNegativeGroupByCounts.csv
predictedCat,predictedCat

Whole_Blood.falsePositiveGroupByCounts.csv
category,category
BLCA,2
BRCA,6
DLBC,1
KIRC,2
LIHC,1
LUAD,1
THCA,1
THYM,1
```

# Best Bang for the buck?
```
LUAD Fn PAAD 14

LUAD FP BRCA 11
        LUSC 11

LUSC FN HNSC 21
        LUAD 11
        PAAD 19

LUSC FP BRAC 14
        HNSC 44
```